# Global indicator project
## Create a sausage buffer for each sample point to identify walkable local neighborhood

This notebook creates sausage buffer to determine local walkable neighborhood (1600m distance of sample points + 50m walkable street network buffer). This is used to calculate local neighborhood/location based indicators. This notebook should run after ```04_create_sample_points```. Indicators are calculated per sample point basis and save in sample point stats dataframe.  

Process: 
1. Load pedestrain street network and sample points from local folder
2. Create sausage buffer:
    - locate closest node to each sample point on network, 
    - create network subgraph with 1600m distance around sample point-nodes and 50m walkable street network buffer
    - iterate over all sample points to obtain all local neighborhood graph
3. Get local neighborhood basic stats using OSMnx: 
    - area in km; intersection density; clean intersection count; distance to nearest node
    - save indicators for each local neighborhood in sample point stats dataframe
  


See [this pull request comment discussion](https://github.com/shiqin-liu/global-indicators/pull/1/commits/9b1b1d524894646c26b8c4120e5cc0d9d8eb490f)

** Buffering technique referring to Forsyth et al [here](https://ij-healthgeographics.biomedcentral.com/articles/10.1186/1476-072X-11-14). Mavoa et al 2018, see [here](https://www.jtlu.org/index.php/jtlu/article/view/1132)   
** load the projected graph, using the projected graph to get network stats so that the ouput unit is m/km

In [81]:
from setup_OSMnetwork import *
from setup_local_analysis import *
from config import *

## 1. Load pedestrain street network and sample points from local folder

In [2]:
## Load pedestrain street network graphml and edge shapefile from local folder
G_proj = get_OSM_G(G_proj_filename, OSM_folder)

In [3]:
# load sample point shapefile
sample_points_gdf = gpd.GeoDataFrame.from_file(samplepoint_filepath)
len(sample_points_gdf)

2331492

In [4]:
#here, I just take 600 points as sample (due to the memory deficiency) to test the scripts 
sample_points = sample_points_gdf.sample(n=600, replace=True, random_state=1)

start = time.time()
task = "create list of urban sample points location for {}".format(placename)
# create list of sample points to iterate over
point_locations = []

for point in sample_points.geometry: 
    point = (point.x, point.y)
    point_locations = point_locations + [point]
    
print('Completed task "{}" with {} points in {:,.2f} seconds'.format(
    task, len(point_locations), time.time() - start)) 


Completed task "create list of urban sample points location for phoenix" with 600 points in 0.85 seconds


## 2. Create sausage buffer

```
#run create buffer analysis
start = time.time()
task = 'Buffer network for {} sample points'.format(len(point_locations))
sausagebuffers = []
for point in point_locations:
    sausagebuffers.append(create_sausage_buffer_gdf(G_proj, point))
print('Completed task "{}" in {:,.2f} seconds'.format(task,time.time() - start)) 
```

## 3. Get sample point neighborhood stats using OSMnx

In [70]:
# initialise a list
start = time.time()
task = "Calculate local neighbourhood measures for {} points".format(len(point_locations))
nh_estimates = []
for point in point_locations:
    nh_estimates.append(analyse_local_nh(G_proj, point))
    

In [71]:
# area in sqkm
area_km = [x['area_sqkm'] for x in nh_estimates]
# clean intersection density per sqkm
clean_intersection_count = [x['stats']['clean_intersection_count'] for x in nh_estimates]
# clean intersection density
clean_intersection_density_km = [x['stats']['clean_intersection_density_km'] for x in nh_estimates]
# dist to the nodes
origin_node_snap_dist = [x['origin_node_snap_dist'] for x in nh_estimates]
# nearest node id
origin_node_id = [x['origin_node_id'] for x in nh_estimates]

#load sample point dataframe and record new stats
sample_points['area_km'] = area_km
sample_points['intct_count'] = clean_intersection_count
sample_points['intct_den']= clean_intersection_density_km
sample_points['node_dist']= origin_node_snap_dist
sample_points['node_id']= origin_node_id


print('Completed task "{}" in {:,.2f} seconds'.format(task,time.time() - start))

Completed task "Calculate local neighbourhood measures for 600 points" in 24,061.89 seconds


In [ ]:
ox.save_gdf_shapefile(sample_points, filename='{studyregion}_urban_sample_points_stats{suffix}'.format(
        studyregion = placename, suffix = suffix), folder=OSM_folder)